In [1]:
# === Importando librerías ===
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pandas as pd

# ===========================
# 1) PROBLEMA DIRECTO (KP)
# ===========================
p1  = 0.1245
g1  = 2e3
mu2 = 0.03
r2  = 0.18
b   = 1e-5
a   = 1.0
g2  = 1e1
mu3 = 10.0
p2  = 5.0
g3  = 1e-1
c   = 0.0297

s1 = 0.045
s2 = 1.0

def dP_dt(P, t):
    x, y, z = P
    dx = c*y - mu2*x + p1*x*z/(g1 + z) + s1
    dy = r2*y*(1 - b*y) - a*x*y/(g2 + y)
    dz = p2*x*y/(g3 + y) - mu3*z + s2
    return [dx, dy, dz]

t = np.linspace(0, 300, 300)
P0 = [1.0, 1.0, 1.0]
datos_sol = odeint(dP_dt, P0, t)
x, y, z = datos_sol[:, 0], datos_sol[:, 1], datos_sol[:, 2]

t_values  = torch.tensor(t, dtype=torch.float32).view(-1, 1)
E_values  = torch.tensor(x, dtype=torch.float32).view(-1, 1)
T_values  = torch.tensor(y, dtype=torch.float32).view(-1, 1)
IL_values = torch.tensor(z, dtype=torch.float32).view(-1, 1)

n_obs = 30
N = t_values.shape[0]
idx_obs = torch.randperm(N)[:n_obs]
idx_obs, _ = torch.sort(idx_obs)

t_obs = t_values[idx_obs]
E_obs, T_obs, IL_obs = E_values[idx_obs], T_values[idx_obs], IL_values[idx_obs]

sigma_E  = 0.05 * E_obs.mean()
sigma_T  = 0.05 * T_obs.mean()
sigma_IL = 0.05 * IL_obs.mean()

E_obs_r  = torch.clamp(E_obs  + sigma_E  * torch.randn_like(E_obs),  min=0.0)
T_obs_r  = torch.clamp(T_obs  + sigma_T  * torch.randn_like(T_obs),  min=0.0)
IL_obs_r = torch.clamp(IL_obs + sigma_IL * torch.randn_like(IL_obs), min=0.0)

# ===========================
# 2) PINN + PROBLEMA INVERSO
# ===========================
class FCN_Ansatz(nn.Module):
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS, E0, T0, IL0):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(nn.Linear(N_INPUT, N_HIDDEN), activation())
        self.fch = nn.Sequential(*[
            nn.Sequential(nn.Linear(N_HIDDEN, N_HIDDEN), activation())
            for _ in range(N_LAYERS - 1)
        ])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
        self.register_buffer("E0",  torch.tensor([[E0]],  dtype=torch.float32))
        self.register_buffer("T0",  torch.tensor([[T0]],  dtype=torch.float32))
        self.register_buffer("IL0", torch.tensor([[IL0]], dtype=torch.float32))

    def forward(self, t):
        N = self.fce(self.fch(self.fcs(t)))
        mask = (1 - torch.exp(-5*t))
        E_hat  = self.E0  + mask * N[:,0:1]
        T_hat  = self.T0  + mask * N[:,1:2]
        IL_hat = self.IL0 + mask * N[:,2:3]
        return torch.cat([E_hat, T_hat, IL_hat], dim=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

t_f = 300
t_physics = torch.tensor((np.polynomial.legendre.leggauss(150)[0] + 1)/2 * t_f,
                         dtype=torch.float32, device=device).reshape(-1, 1).requires_grad_(True)
t_test = torch.linspace(0, t_f, 300, device=device).view(-1, 1)

p1_t  = torch.tensor(p1, dtype=torch.float32, device=device)
g1_t  = torch.tensor(g1, dtype=torch.float32, device=device)
mu2_t = torch.tensor(mu2, dtype=torch.float32, device=device)
b_t   = torch.tensor(b,  dtype=torch.float32, device=device)
g2_t  = torch.tensor(g2, dtype=torch.float32, device=device)
mu3_t = torch.tensor(mu3, dtype=torch.float32, device=device)
p2_t  = torch.tensor(p2, dtype=torch.float32, device=device)
g3_t  = torch.tensor(g3, dtype=torch.float32, device=device)
s1_t  = torch.tensor(s1, dtype=torch.float32, device=device)
s2_t  = torch.tensor(s2, dtype=torch.float32, device=device)
r2_t  = torch.tensor(r2, dtype=torch.float32, device=device)

t_values = t_values.to(device)
E_values = E_values.to(device)
T_values = T_values.to(device)
IL_values = IL_values.to(device)
t_obs = t_obs.to(device)
E_obs = E_obs.to(device)
T_obs = T_obs.to(device)
IL_obs = IL_obs.to(device)
sigma_E  = sigma_E.to(device)
sigma_T  = sigma_T.to(device)
sigma_IL = sigma_IL.to(device)

# ===========================================
# 3) ESTUDIO DE PACIENTES
# ===========================================
num_pacientes = 50
epochs = 30000

resultados = []  # [paciente, c_est, a_est]

for paciente in range(1, num_pacientes+1):
    print(f"\n=== Entrenando paciente {paciente}/{num_pacientes} ===")

    PINN = FCN_Ansatz(1, 3, 32, 3, E0=1.0, T0=1.0, IL0=1.0).to(device)

    c_t  = nn.Parameter(torch.rand(1, device=device))
    a_t  = nn.Parameter(torch.rand(1, device=device))

    optimiser = torch.optim.Adam(list(PINN.parameters()) + [c_t, a_t],
                                 lr=1e-3, weight_decay=1e-4)

    E_obs_r_pt  = torch.clamp(E_obs  + sigma_E  * torch.randn_like(E_obs),  min=0.0)
    T_obs_r_pt  = torch.clamp(T_obs  + sigma_T  * torch.randn_like(T_obs),  min=0.0)
    IL_obs_r_pt = torch.clamp(IL_obs + sigma_IL * torch.randn_like(IL_obs), min=0.0)

    for i in range(epochs+1):
        optimiser.zero_grad()
        u, v, w = PINN(t_physics).split(1, dim=1)
        dudt = torch.autograd.grad(u, t_physics, torch.ones_like(u), create_graph=True)[0]
        dvdt = torch.autograd.grad(v, t_physics, torch.ones_like(v), create_graph=True)[0]
        dwdt = torch.autograd.grad(w, t_physics, torch.ones_like(w), create_graph=True)[0]

        c_eff  = torch.nn.functional.softplus(c_t)
        a_eff  = torch.nn.functional.softplus(a_t)

        rhs_E  = c_eff*v - mu2_t*u + (p1_t*u*w)/(g1_t + w) + s1_t
        rhs_T  = r2_t*v*(1 - b_t*v) - a_eff*u*v/(g2_t + v)
        rhs_IL = (p2_t*u*v)/(g3_t + v) - mu3_t*w + s2_t

        loss_phys = ((dudt - rhs_E)**2).mean() \
                  + ((dvdt - rhs_T)**2).mean() \
                  + ((dwdt - rhs_IL)**2).mean()

        u_obs_pred, v_obs_pred, w_obs_pred = PINN(t_obs).split(1, dim=1)
        loss_obs = ((u_obs_pred - E_obs_r_pt)**2).mean() \
                 + ((v_obs_pred - T_obs_r_pt)**2).mean() \
                 + ((w_obs_pred - IL_obs_r_pt)**2).mean()

        c0 = 0.0297
        loss_prior = ((c_eff - c0)/0.033).pow(2).mean()  # 0.05 es la varianza deseada
        loss = 0.1*loss_phys + loss_obs + 1e-2*loss_prior
        loss.backward()
        optimiser.step()

        if i % 5000 == 0:
            print(f"Iter {i:6d} | Loss={loss.item():.6f} | c={c_eff.item():.6f} | a={a_eff.item():.6f}")

    resultados.append([paciente, c_eff.item(), a_eff.item()])

# ===========================
# 4) TABLAS DE RESULTADOS
# ===========================
df_pacientes = pd.DataFrame(resultados, columns=["Paciente", "c_estimado", "a_estimado"])
pd.options.display.float_format = '{:,.6f}'.format

display(df_pacientes.style.hide(axis="index").set_properties(**{'text-align': 'center'}))

c_mean = df_pacientes["c_estimado"].mean()
a_mean = df_pacientes["a_estimado"].mean()

c_median = df_pacientes["c_estimado"].median()
a_median = df_pacientes["a_estimado"].median()

c_std = df_pacientes["c_estimado"].std()
a_std = df_pacientes["a_estimado"].std()

c_err_pct = 100.0 * abs(c_mean - c) / c
a_err_pct = 100.0 * abs(a_mean - a) / a

df_resumen = pd.DataFrame({
    "Parámetro": ["c", "a"],
    "Valor real": [c, a],
    "Media (estimada)": [c_mean, a_mean],
    "Mediana (estimada)": [c_median, a_median],
    "Desviación típica": [c_std, a_std],
    "Error (%)": [c_err_pct, a_err_pct]
})

display(df_resumen.style.hide(axis="index").set_properties(**{'text-align': 'center'}))



=== Entrenando paciente 1/50 ===
Iter      0 | Loss=11.168213 | c=0.923339 | a=0.837251
Iter   5000 | Loss=0.116149 | c=0.136688 | a=0.928111
Iter  10000 | Loss=0.011320 | c=0.047220 | a=0.929647
Iter  15000 | Loss=0.008188 | c=0.030461 | a=0.931334
Iter  20000 | Loss=0.009033 | c=0.030363 | a=0.933260
Iter  25000 | Loss=0.007391 | c=0.030363 | a=0.932673
Iter  30000 | Loss=0.006817 | c=0.030362 | a=0.933919

=== Entrenando paciente 2/50 ===
Iter      0 | Loss=16.511631 | c=1.275840 | a=0.955219
Iter   5000 | Loss=0.261830 | c=0.191766 | a=0.977221
Iter  10000 | Loss=0.021619 | c=0.060722 | a=0.976410
Iter  15000 | Loss=0.013120 | c=0.031485 | a=0.977086
Iter  20000 | Loss=0.011932 | c=0.030332 | a=0.979696
Iter  25000 | Loss=0.011432 | c=0.030331 | a=0.979699
Iter  30000 | Loss=0.011405 | c=0.030331 | a=0.979122

=== Entrenando paciente 3/50 ===
Iter      0 | Loss=21.750139 | c=1.291828 | a=0.847166
Iter   5000 | Loss=0.262139 | c=0.194338 | a=0.946507
Iter  10000 | Loss=0.018481 | c

Paciente,c_estimado,a_estimado
1,0.030362,0.933919
2,0.030331,0.979122
3,0.030357,0.949518
4,0.030330,0.972094
5,0.030329,0.961702
6,0.030326,1.002884
7,0.030310,1.014302
8,0.030298,1.020090
9,0.030317,0.997961
10,0.030341,0.922955


Parámetro,Valor real,Media (estimada),Mediana (estimada),Desviación típica,Error (%)
c,0.029700,0.030335,0.030335,0.000019,2.137712
a,1.000000,0.969261,0.966624,0.028949,3.073873
